# Module 07: Integrate Azure Cosmos DB SQL API with Azure services

- [[Learning path]](https://docs.microsoft.com/en-us/learn/paths/integrate-azure-cosmos-db-sql-api-azure-services/?ns-enrollment-type=Collection&ns-enrollment-id=1k8wcz8zooj2nx)
- [[Lab]](https://microsoftlearning.github.io/dp-420-cosmos-db-dev/instructions/13-change-feed.html): Process change feed events using the Azure Cosmos DB SQL API SDK
- [[Lab]](https://microsoftlearning.github.io/dp-420-cosmos-db-dev/instructions/14-functions.html): Process Azure Cosmos DB SQL API data using Azure Functions
- [[Lab]](https://microsoftlearning.github.io/dp-420-cosmos-db-dev/instructions/15-cognitive-search.html): Search data using Azure Cognitive Search and Azure Cosmos DB SQL API

## Demo setup

In [ ]:
Connect-AzAccount
Set-AzContext -Subscription "b895a719-7034-411a-9944-ff196d1f450f"
$connectionString = (Get-AzCosmosDBAccountKey -ResourceGroupName rg-dp-420 -Name cosmos-dp-420-sql-provisioned -Type "ConnectionStrings")["Primary SQL Connection String"]
$primaryMasterKey = (Get-AzCosmosDBAccountKey -ResourceGroupName rg-dp-420 -Name cosmos-dp-420-sql-provisioned -Type "Keys")["PrimaryMasterKey"]
$documentEndpoint = (Get-AzCosmosDBAccount -ResourceGroupName rg-dp-420 -Name cosmos-dp-420-sql-provisioned).DocumentEndpoint

In [ ]:
cosmicworks --endpoint $documentEndpoint --key $primaryMasterKey --datasets product

In [ ]:
#r "nuget: Newtonsoft.Json, 13.0.1"
#r "nuget: Microsoft.Azure.Cosmos , 3.26.0"

#!share --from pwsh connectionString
#!share --from pwsh primaryMasterKey
#!share --from pwsh documentEndpoint

In [ ]:
using Microsoft.Azure.Cosmos;
using System;
using System.Collections.Generic;

CosmosClient client = new (connectionString);
Database database = client.GetDatabase("cosmicworks");
Container container = database.GetContainer("products");

public class Product
{
	public string id { get; set; }
	public string categoryId { get; set; }
	public string categoryName { get; set; }
	public string sku { get; set; }
	public string name { get; set; }
	public string description { get; set; }
	public double price { get; set; }
}

## Consume an Azure Cosmos DB SQL API change feed using the SDK

### Understand change feed features in the SDK

The .NET SDK for Azure Cosmos DB SQL API ships with a change feed processor that simplifies the task of reading changes from the feed.

| **Component** | **Description** |
| --- | --- |
| **Monitored container** | This container is monitored for any insert or update operations. These changes are then reflected in the feed. |
| **Lease container** | The lease container serves as a storage mechanism to manage state across multiple change feed consumers (clients). |
| **Host** | The host is a client application instance that listens for and reacts to changes from the change feed. |
| **Delegate** | The delegate is code within the client application that will implement business logic for each batch of changes. |

Prior to using the change feed processor, you should create a lease container that you will reference when configuring the processor.

### Implement a delegate for the change feed processor

For the change feed processor, the library expects a delegate of type ChangesHandler<>. This delegate will handle a list of changes on the feed.

In [ ]:
// Create an instance for both the source and lease container.
Container sourceContainer = await database.CreateContainerIfNotExistsAsync("products", "/categoryId", 400); 
Container leaseContainer = await database.CreateContainerIfNotExistsAsync("productslease", "/partitionKey", 400);

In [ ]:
// The delegate includes two parameters; a read-only list of changes and a cancellation token.
using System;
using System.Collections.Generic;
using System.IO;
using System.Threading;
using System.Threading.Tasks;
using Microsoft.Azure.Cosmos;
using Microsoft.Azure.Cosmos.Fluent;
using static Microsoft.Azure.Cosmos.Container;

ChangesHandler<Product> changeHandlerDelegate = async ( 
    IReadOnlyCollection<Product> changes, 
    CancellationToken cancellationToken ) => 
        { 
            // A foreach loop is used to iterate through the current batch of changes, 
            // and print each item to the console window
            foreach(Product product in changes) 
            { 
                await Console.Out.WriteLineAsync($"Detected Operation:\t[{product.id}]\t{product.name}"); 
                // Do something else with each change 
            } 
         };

### Implement the change feed processor

The processor needs to be built with a source and lease container, and then needs to be started.

In [ ]:
ChangesHandler<Product> changeHandlerDelegate = async ( 
    IReadOnlyCollection<Product> changes, 
    CancellationToken cancellationToken ) => 
        { 
            // A foreach loop is used to iterate through the current batch of changes, 
            // and print each item to the console window
            foreach(Product product in changes) 
            { 
                await Console.Out.WriteLineAsync($"Detected Operation:\t[{product.id}]\t{product.name}"); 
                // Do something else with each change 
            } 
         };

// Use the GetChangeFeedProcessorBuilder method from the source container instance to create a builder.
var builder = sourceContainer.GetChangeFeedProcessorBuilder<Product>( 
    processorName: "productItemProcessor", 
    onChangesDelegate: changeHandlerDelegate // Delegate defined in previous slide 
);

// Build the change feed processor with the defined builder and lease container
ChangeFeedProcessor processor = builder 
    .WithInstanceName("desktopApplication") 
    .WithLeaseContainer(leaseContainer) 
    .Build();

// Run processor asynchronously.
await processor.StartAsync(); 

// Wait while processor handles items 
await Task.Delay(10000);

// Terminate processor asynchronously.
await processor.StopAsync();

In [ ]:
cosmicworks --endpoint $documentEndpoint --key $primaryMasterKey --datasets product

## Handle events with Azure Functions and Azure Cosmos DB SQL API change feed

## Search Azure Cosmos DB SQL API data with Azure Cognitive Search